In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


## Load data


In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/lily/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/lily/test.csv')

output_map = {'Animal Diseases': 0,
 'Bacterial Infections and Mycoses': 3,
 'Cardiovascular Diseases': 1,
 'Chemically-Induced Disorders': 2,
 'Congenital Hereditary and Neonatal Diseases and Abnormalities': 4,
 'Digestive System Diseases': 5,
 'Disorders of Environmental Origin': 22,
 'Endocrine System Diseases': 6,
 'Eye Diseases': 7,
 'Female Urogenital Diseases and Pregnancy Complications': 8,
 'Hemic and Lymphatic Diseases': 9,
 'Immune System Diseases': 10,
 'Male Urogenital Diseases': 11,
 'Musculoskeletal Diseases': 24,
 'Neoplasms': 12,
 'Nervous System Diseases': 13,
 'Nutritional and Metabolic Diseases': 14,
 'Occupational Diseases': 15,
 'Otorhinolaryngologic Diseases': 16,
 'Parasitic Diseases': 17,
 'Pathological Conditions and Signs and Symptoms': 18,
 'Respiratory Tract Diseases': 19,
 'Skin and Connective Tissue Diseases': 25,
 'Stomatognathic Diseases': 20,
 'Virus Diseases': 21,
 'Wounds and Injuries': 23}

train['categories'] = train['categories'].map(output_map)
test['categories'] = test['categories'].map(output_map)

In [ ]:
## import packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.utils import np_utils
import contractions
import re
import tqdm
import unicodedata

In [ ]:
!pip install contractions

     |████████████████████████████████| 245kB 18.4MB/s 
     |████████████████████████████████| 317kB 42.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81695 sha256=904135168a7136ab5aa8d9ef4f5b369aef6b80621168a7e28d0dac0aa8d19cf6
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


Basic Text Pre-processing
We do minimal text pre-processing here since we are using deep learning models and not count-based methods. Steps include the following:



Converting accented characters
*   Removal of stop words
*   Fixing contractions
*   Removing special characters
*   Converting accented characters

Note : For some models we don't use any pre-processing like BERT!

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', ' ', doc, re.I|re.A)
        doc = doc.strip()  
        norm_docs.append(doc)
    return norm_docs

In [ ]:
%%time
train_content = pre_process_corpus(list(train['abstract']))
test_content = pre_process_corpus(list(test['abstract']))

100%|██████████| 10862/10862 [00:01<00:00, 5435.49it/s]

CPU times: user 10 s, sys: 92.1 ms, total: 10.1 s
Wall time: 10.1 s


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
# tokenize the inputs
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_content)
# get the word to index mapping for input language
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 122083 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(train_content)
# pad the sequences
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (43916, 250)


In [ ]:
## one hot encoding of output labels
Y  = np_utils.to_categorical(train['categories'])
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (43916, 26)


In [ ]:
## tokenize and pad the test data
test_x = tokenizer.texts_to_sequences(test_content)
test_x = pad_sequences(test_x, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', test_x.shape)

Shape of data tensor: (10862, 250)


In [ ]:
test_Y  = np_utils.to_categorical(test['categories'])
print('Shape of label tensor:', test_Y.shape)

Shape of label tensor: (10862, 26)


In [ ]:
### save preprocessed data to disk
np.save('/content/gdrive/My Drive/lily/keras_train_data.npy',X)
np.save('/content/gdrive/My Drive/lily/keras_test_data.npy',test_x)
np.save('/content/gdrive/My Drive/lily/keras_train_label.npy',Y)
np.save('/content/gdrive/My Drive/lily/keras_test_label.npy',test_Y)

## Build Model Architecture


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(26, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 250, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 26)                2626      
Total params: 5,083,026
Trainable params: 5,083,026
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
### training
epochs = 100
batch_size = 64
history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_data=(test_x, test_Y))

Epoch 1/100
687/687 [==============================] - 648s 944ms/step - loss: 2.6731 - accuracy: 0.2161 - val_loss: 2.2497 - val_accuracy: 0.3723
Epoch 2/100
687/687 [==============================] - 642s 934ms/step - loss: 1.8787 - accuracy: 0.4919 - val_loss: 1.7103 - val_accuracy: 0.5526
Epoch 3/100
687/687 [==============================] - 637s 928ms/step - loss: 1.4203 - accuracy: 0.6172 - val_loss: 1.6132 - val_accuracy: 0.5943
Epoch 4/100
687/687 [==============================] - 645s 938ms/step - loss: 1.1533 - accuracy: 0.6990 - val_loss: 1.4978 - val_accuracy: 0.6279
Epoch 5/100
687/687 [==============================] - 682s 993ms/step - loss: 0.9363 - accuracy: 0.7547 - val_loss: 1.4734 - val_accuracy: 0.6375
Epoch 6/100
687/687 [==============================] - 685s 997ms/step - loss: 0.7768 - accuracy: 0.7971 - val_loss: 1.3972 - val_accuracy: 0.6653
Epoch 7/100
687/687 [==============================] - 682s 992ms/step - loss: 0.6209 - accuracy: 0.8378 - val_loss: 1

KeyboardInterrupt: ignored

In [ ]:
## save the model
model.save('/content/gdrive/My Drive/lily/keras_model.h5')